> `COALESCE(expression1, expression2, ..., expressionN)` me devuelve la expresión final en caso de que ambos sean nulos, por ejemplo, `COALESCE(tabla1, tabla2, 'valor desconocido') as tabla_final`, en este caso si tabla1 es null, pero tabla 2 no, me pone el valor de tabla1, si ambos son nulos lo guarda como valor desconocido. Si se usa con una sola tabla es igual.

> `IFNULL(column_name, value_if_null)` funciona igual solo que es una sola tabla o expresión

> Case en where
```sql
SELECT 
  actor, 
  character, 
  platform
FROM marvel_avengers
WHERE 
  CASE 
    WHEN platform = 'Instagram' THEN followers >= 500000
    WHEN platform = 'Twitter' THEN followers >= 200000
    ELSE followers >= 100000
  END;
```

## Bugbuster

Cuantos productos (únicos) tiene cada categoria

```sql
select category, count(DISTINCT product)
from product_spend 
group by category
```

Usar case, para los actores con tantos likes crear una columna llamada popularity que me los categorice como super likes, good likes y low likes.

```sql
select actor, character, platform, avg_likes,
CASE
  when avg_likes >= 15000 then 'Super Likes'
  when avg_likes between 5000 and 14999 then 'Good Likes'
  else 'Low Likes'
end as popularity
from marvel_avengers
order by avg_likes desc
```

productos por categoria

```sql
select category, count(DISTINCT product)
from product_spend 
group by category
```

top 3 del total de ordenes por ciudad, donde el status de la orden esté completo

```sql
select users.city, count(status) as total_orders
from trades
join users on trades.user_id = users.user_id
where status = 'Completed'
group by users.city
order by total_orders desc
limit 3
```

Escribe una consulta que devuelva los ID de las páginas de Facebook que tienen cero me gusta. La salida debe estar ordenada de forma ascendente en función de los ID de página. (Los **page_id** que salen en la tabla **pages** no salen en la tabla **page_likes** si no se registran likes, entonces al unir con el join esas filas quedan vacias, usamos el where para filtrar las de donde la tabla page_likes sea null)

```sql
SELECT pages.page_id FROM pages
left join page_likes on pages.page_id = page_likes.page_id
where page_likes is NULL
order by page_id asc
```

Dadas dos tablas, advertiser (con información de anunciantes y su estado de pago) y daily_pay (que solo contiene registros de pagos recientes), se requiere una consulta SQL para actualizar el estado de pago de los anunciantes. El estado puede ser NEW, EXISTING, CHURN o RESURRECT, dependiendo de si han realizado pagos previamente y si hicieron un pago reciente. La salida debe mostrar el user_id y su nuevo estado, ordenados por user_id.

```sql
SELECT 
  COALESCE(advertiser.user_id, daily_pay.user_id) as user_id,
  CASE
    WHEN paid is null then 'CHURN'
    when paid is not null and status IN('EXISTING','NEW','RESURRECT') then 'EXISTING'
    when paid is not null and status = 'CHURN' then 'RESURRECT'
    else 'NEW'
  end as "new_status"
FROM advertiser
full outer join daily_pay on daily_pay.user_id = advertiser.user_id
order by user_id
```

Para los usuarios que han posteado al menos una vez, escribir una query que encuentre el número de dias entre la primera publicación y la última, en el año 2021.

```sql
SELECT user_id,
EXTRACT(DAY FROM max(post_date) - min(post_date)) as days_between
from posts
where extract(year from post_date) = 2021
group by user_id
having count(user_id) >= 2
```

Escribir un query que me muestre el id de los usuarios que confirmaron el email el siguiente dia y no el mismo dia.

```sql
SELECT user_id FROM emails
left join texts on texts.email_id=emails.email_id
where action_date = signup_date + INTERVAL '1 day'
```